In [1]:
import pandas as pd
import scipy.stats as ss
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.api import anova_lm
from statsmodels.stats.multicomp import (pairwise_tukeyhsd, MultiComparison)
import pingouin as pg
import seaborn as sns

In [42]:
from IPython.display import display, HTML
display(HTML("""
<style>
.jp-Notebook {
    max-width: 500px;
    margin: auto;
}
.jp-Cell-outputArea {
    max-width: 500px;
}
</style>
"""))

In [43]:
df = pd.read_csv('7_cars.csv')

In [48]:
df.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   car_ID            205 non-null    int64  
 1   symboling         205 non-null    int64  
 2   CarName           205 non-null    object 
 3   fueltype          205 non-null    object 
 4   aspiration        205 non-null    object 
 5   doornumber        205 non-null    object 
 6   carbody           205 non-null    object 
 7   drivewheel        205 non-null    object 
 8   enginelocation    205 non-null    object 
 9   wheelbase         205 non-null    float64
 10  carlength         205 non-null    float64
 11  carwidth          205 non-null    float64
 12  carheight         205 non-null    float64
 13  curbweight        205 non-null    int64  
 14  enginetype        205 non-null    object 
 15  cylindernumber    205 non-null    object 
 16  enginesize        205 non-null    int64  
 1

In [5]:
df.describe()

,car_ID,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
count,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000
mean,103.000000,0.834146,98.756585,174.049268,65.907805,53.724878,2555.565854,126.907317,3.329756,3.255415,10.142537,104.117073,5125.121951,25.219512,30.751220,13276.710571
std,59.322565,1.245307,6.021776,12.337289,2.145204,2.443522,520.680204,41.642693,0.270844,0.313597,3.972040,39.544167,476.985643,6.542142,6.886443,7988.852332
min,1.000000,-2.000000,86.600000,141.100000,60.300000,47.800000,1488.000000,61.000000,2.540000,2.070000,7.000000,48.000000,4150.000000,13.000000,16.000000,5118.000000
25%,52.000000,0.000000,94.500000,166.300000,64.100000,52.000000,2145.000000,97.000000,3.150000,3.110000,8.600000,70.000000,4800.000000,19.000000,25.000000,7788.000000
50%,103.000000,1.000000,97.000000,173.200000,65.500000,54.100000,2414.000000,120.000000,3.310000,3.290000,9.000000,95.000000,5200.000000,24.000000,30.000000,10295.000000
75%,154.000000,2.000000,102.400000,183.100000,66.900000,55.500000,2935.000000,141.000000,3.580000,3.410000,9.400000,116.000000,5500.000000,30.000000,34.000000,16503.000000
max,205.000000,3.000000,120.900000,208.100000,72.300000,59.800000,4066.000000,326.000000,3.940000,4.170000,23.000000,288.000000,6600.000000,49.000000,54.000000,45400.000000


Создайте новый признак – марку автомобиля (company). Машины каких производителей встречаются в датасете? Далее исправьте названия и проверьте изменения.

In [ ]:
# мой вариант извлечения названия компании из названия автомобиля (слишком сложно)
df['company'] = df["CarName"].str.extract(r'^([^ ]+)')

In [49]:
df['company'] = df['CarName'].str.split(' ').str[0]

In [ ]:
# исправление опечаток в названиях компаний автомобилей ручным методом
df['company'] = df['company'].replace({'maxda': 'mazda', 'Nissan': 'nissan', 'porcshce': 'porsche', 'toyouta': 'toyota', 'vokswagen': 'volkswagen', 'vw': 'volkswagen'})

In [50]:
df['company'] = df['company'].str.lower()

In [52]:
def fix_company(to_rep, how_rep):
    df['company'] = df['company'].str.replace(to_rep, how_rep)

In [51]:
fix_company('maxda', 'mazda')
fix_company('porcshce', 'porsche')
fix_company('toyouta', 'toyota')
fix_company('vokswagen', 'volkswagen')
fix_company('vw', 'volkswagen')

In [53]:
df['company'].nunique()

22

Оставьте только часть предикторов, после чего посчитайте корреляцию между price и другими переменными

In [54]:
cols_to_keep = ['company', 'fueltype', 'aspiration', 'carbody', 'drivewheel', 'enginetype', 'cylindernumber', 'carlength', 'carwidth', 'curbweight', 'enginesize', 'wheelbase', 'boreratio', 'horsepower', 'price']

In [55]:
cars = df[cols_to_keep]
cars.head()

,company,fueltype,aspiration,carbody,drivewheel,enginetype,cylindernumber,carlength,carwidth,curbweight,enginesize,wheelbase,boreratio,horsepower,price
0,alfa-romero,gas,std,convertible,rwd,dohc,four,168.8,64.1,2548,130,88.6,3.47,111,13495.0
1,alfa-romero,gas,std,convertible,rwd,dohc,four,168.8,64.1,2548,130,88.6,3.47,111,16500.0
2,alfa-romero,gas,std,hatchback,rwd,ohcv,six,171.2,65.5,2823,152,94.5,2.68,154,16500.0
3,audi,gas,std,sedan,fwd,ohc,four,176.6,66.2,2337,109,99.8,3.19,102,13950.0
4,audi,gas,std,sedan,4wd,ohc,five,176.6,66.4,2824,136,99.4,3.19,115,17450.0


In [56]:
cars.select_dtypes(include='number').corr()

,carlength,carwidth,curbweight,enginesize,wheelbase,boreratio,horsepower,price
carlength,1.000000,0.841118,0.877728,0.683360,0.874587,0.606454,0.552623,0.682920
carwidth,0.841118,1.000000,0.867032,0.735433,0.795144,0.559150,0.640732,0.759325
curbweight,0.877728,0.867032,1.000000,0.850594,0.776386,0.648480,0.750739,0.835305
enginesize,0.683360,0.735433,0.850594,1.000000,0.569329,0.583774,0.809769,0.874145
wheelbase,0.874587,0.795144,0.776386,0.569329,1.000000,0.488750,0.353294,0.577816
boreratio,0.606454,0.559150,0.648480,0.583774,0.488750,1.000000,0.573677,0.553173
horsepower,0.552623,0.640732,0.750739,0.809769,0.353294,0.573677,1.000000,0.808139
price,0.682920,0.759325,0.835305,0.874145,0.577816,0.553173,0.808139,1.000000


In [57]:
corr_mat = cars.select_dtypes(include='number').corr()
corr_mat['price'].sort_values(ascending=False).round(2)

price         1.00
enginesize    0.87
curbweight    0.84
horsepower    0.81
carwidth      0.76
carlength     0.68
wheelbase     0.58
boreratio     0.55
Name: price, dtype: float64

Последний шаг в подготовке данных: линейная регрессия в python не справляется с категориальными переменными (типом object в pandas), поэтому давайте применим функцию под названием pd.get dummies(). Она создаёт т.н. фиктивные переменные на основе изначальных категорий, представленные в виде 0 и 1. Официально такое преобразование называется One-Hot Encoding.  
Для начала примените её только на категориальных переменных:  
df_dummy = pd.get_dummies(data=cars[[список_столбцов_типа_object]], drop_first = True)  
Теперь примените её ко всему датафрейму (с тем же набором колонок, что и на предыдущем шаге). Не забудьте сохранить результат, он нам ещё понадобится :)

In [58]:
cars.dtypes

company            object
fueltype           object
aspiration         object
carbody            object
drivewheel         object
enginetype         object
cylindernumber     object
carlength         float64
carwidth          float64
curbweight          int64
enginesize          int64
wheelbase         float64
boreratio         float64
horsepower          int64
price             float64
dtype: object

In [62]:
cars_dummy = pd.get_dummies(data=cars[['company', 'fueltype', 'aspiration', 'carbody', 'drivewheel', 'enginetype', 'cylindernumber']], drop_first=True)
cars_dummy.head()

,company_audi,company_bmw,company_buick,company_chevrolet,company_dodge,company_honda,company_isuzu,company_jaguar,company_mazda,company_mercury,...,enginetype_ohc,enginetype_ohcf,enginetype_ohcv,enginetype_rotor,cylindernumber_five,cylindernumber_four,cylindernumber_six,cylindernumber_three,cylindernumber_twelve,cylindernumber_two
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,True,False,False,False
3,True,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,True,False,False,False,False
4,True,False,False,False,False,False,False,False,False,False,...,True,False,False,False,True,False,False,False,False,False


In [63]:
cars_lr = pd.get_dummies(data=cars, drop_first=True)
cars_lr.head()

,carlength,carwidth,curbweight,enginesize,wheelbase,boreratio,horsepower,price,company_audi,company_bmw,...,enginetype_ohc,enginetype_ohcf,enginetype_ohcv,enginetype_rotor,cylindernumber_five,cylindernumber_four,cylindernumber_six,cylindernumber_three,cylindernumber_twelve,cylindernumber_two
0,168.8,64.1,2548,130,88.6,3.47,111,13495.0,False,False,...,False,False,False,False,False,True,False,False,False,False
1,168.8,64.1,2548,130,88.6,3.47,111,16500.0,False,False,...,False,False,False,False,False,True,False,False,False,False
2,171.2,65.5,2823,152,94.5,2.68,154,16500.0,False,False,...,False,False,True,False,False,False,True,False,False,False
3,176.6,66.2,2337,109,99.8,3.19,102,13950.0,True,False,...,True,False,False,False,False,True,False,False,False,False
4,176.6,66.4,2824,136,99.4,3.19,115,17450.0,True,False,...,True,False,False,False,True,False,False,False,False,False
